In [9]:
import torch
import torch.utils.data

In [12]:
from Coach import Coach
from Game import GameImp as Game
from NN import NNetWrapper as nn

ImportError: cannot import name 'cards' from 'fireplace2' (unknown location)

In [ ]:
class Args:
    def __init__(self):
        self.n_in = 34 * 16
        self.n_out = 21 * 18

        self.epochs = 50
        self.batch_size = 128
        self.lr = 1e-4

        self.arenaCompare = 40
        self.numIters = 10  # 25
        self.numEps = 25
        self.maxlenOfQueue = 100000
        self.numMCTS = 100  # 1000
        self.numItersForTrainExamplesHistory = 20

        self.tempThreshold = 15
        self.updateThreshold = 0.6

        self.ngpu = 0
        self.device = torch.device("cuda:0" if (torch.cuda.is_available() and self.ngpu > 0) else "cpu")

        self.checkpoint = './temp/'
        self.load_model = False
        self.load_folder_file = ('. / temp / ', 'temp.pth.tar')
        self.fireplace_log_enabled = True

In [11]:
args = Args()
g = Game()
nnet = nn(args)

if args.load_model:
    nnet.load_checkpoint(args.load_folder_file[0], args.load_folder_file[1])

c = Coach(g, nnet, args)
if args.load_model:
    print("Load trainExamples from file")
    c.load_train_examples()
c.learn()

NameError: name 'Game' is not defined

In [28]:
game = Game()
cur_game = game.init_game()
game.mulligan_choice()

print(game.get_valid_moves())

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [33]:
game.get_action((3,0))

InvalidAction


In [18]:
import functools
import numpy as np

In [28]:
l1 = np.array([[10, 20], [30, 40, 50]]) 
l2 = np.array([[10, 20], [30, 40, 50]]) 

if not functools.reduce(lambda x, y : (x & y).all(), map(lambda p, q: p == q,l1,l2), True): 
    print ("The lists l1 and l2 are the same") 
else: 
    print ("The lists l1 and l2 are not the same")


AttributeError: 'bool' object has no attribute 'all'

In [ ]:
if not functools.reduce(lambda x, y: (x & y).all(), map(lambda p, q: p == q, node.state, game.get_state()), True):
    print("States are not the same")
    node.state = game.get_state()
    node.policy, node.value = self.nnet.predict(node.state)
    node.policy.shape = (21, 18)
    node.policy = node.policy * node.valids
    sum_ps_s = np.sum(node.policy)
    if sum_ps_s > 0:
        node.policy /= sum_ps_s  # renormalize
    else:
        print("All valid moves were masked, do workaround.")
        node.policy = node.policy + node.valids
        node.policy /= np.sum(node.policy)